Entity Resolution
=========

This provides an overview of why and how Entity Resolution is becoming an important 
discipline in [Computer Science and Data Science](http://www.datacommunitydc.org/blog/2013/08/entity-resolution-for-big-data). This notebook explores why we need entity resolution and how to do it. Brief explanations
are given regarding commerical options. Open source options are also discussed. Some of these open
source options are demonstrated using Python.

## Why do we need entity resolution?

When doing a statistical analysis, you need to first identify your units of analysis. 
Borrowing from Allison ([Multiple Regression: A Primer](http://www.amazon.com/Multiple-Regression-Research-Methods-Statistics/dp/0761985336), p. 7):

> To do a regression analysis, you first need a set of cases (also called units of analysis or observations).
> In the social sciences, the cases are most often persons, but they could also be organizations, countries, or
> other groups. In economics, the cases are sometimes units of time, like years or quarters. For each case, you
> need measurements on all of the variables in the regession equation. 

Often, our data does not come to us in one table; ready for analysis. In this situation, we also need to be
aware of database concepts.In web development, we can say that we're concerned with defining the database 
model (see the Python web framework Django for [their model specifications](https://docs.djangoproject.com/en/1.8/topics/db/models/)). In the case of Entity Resolution, we are concerned with two concepts:

1. Primary Keys
1. Foreign Keys

In Django, each model requires exactly one field to have a primary key. A foreign key specifies a one-to-many 
relationship in Django. They have separate field types for one-to-one and many-to-many relationships between models. For our purposes we will exclude many-to-many relationships and include one-to-one relationships in our discussion of the Foreign key. The following is a demonstration from the [Django Documenation of the Foreign key](https://docs.djangoproject.com/en/1.8/ref/models/fields/#django.db.models.ForeignKey). 

``` Python
from django.db import models

class Car(models.Model):
    manufacturer = models.ForeignKey('Manufacturer')
    # ...

class Manufacturer(models.Model):
    # ...
    pass
```

We can see that each type of car could have multiple manufacturers. Several car manufacturers make mid-size sedans. Those manufacturers are reflected in the above Django model. The database models are not specific to Python but it does help to have concrete examples with easily accessible documentation.

When conducting a statistical analysis, we want to run our model on one table. Before conducting the analysis, we must be sure of the following:

1. Each unit of analysis occurs only once 
1. Each unit of analysis includes variables that have been correctly mapped from their original tables
1. Primary and Foreign keys must be present and unique

Entity Resolution is a tool to define primary and foreign keys when these relationships are not previously well defined. For example, let's say people start using the web application described above. They populate the models with cars and
the manufacturers who produce them. The developers did not include standard names for cars and manufacturers because
they were unsure of what the future may hold. As the data scientist arrives on scene, they find that people have spelled
the manufacturer "GMC" in a variety of ways like "General Motors Corporation" and "General Motors". People have also 
mispelled the manufacturer name. Actually, they have done this across much of the manufacturer and car information. 

Given the state of this manufacturer/car data, the data scientist is no longer able to ascertain their unit of analysis. They must recreate primary and foreign keys before they can perform their statistical analysis. 

  
**TL;DR** We can't begin our analysis unless we have defined and identified quantifiable units of analysis. Data often shows up in multiple tables, we need to know about primary and foreign keys. We want to run our statistical analysis on a single table. We must determine primary and foreign keys before we can manipulate our data to perform the statistical analysis. Entity resolution can be used to establish primary and foreign keys when those relationships are not previously well defined.




## How can we accomplish entity resolution?

We have established that entity resolution is a way for the data scientist to restablish primary and foreign keys once people have inputted information which makes those relationships ambiguous from a statistical analysis standpoint. Here is the same goal restated by the [Stanford Entity Resolution Framework (SERF)](http://infolab.stanford.edu/serf/)Project.

> The goal of the SERF project is to develop a generic infrastructure for Entity Resolution (ER). ER (also known as
> deduplication, or record linkage) is an important information integration problem: The same "real-world entities"
> (e.g., customers, or products) are referred to in different ways in multiple data records. For instance, two records
> on the same person may provide different name spellings, and addresses may differ. The goal of ER is to "resolve"
> entities, by identifying the records that represent the same entity and reconciling them to obtain one record per
> entity.

You can accomplish entity resolution by:

1. Hiring a company
1. Doing it yourself
1. A little of both

We will explore each option. 

### Hire a company

[Basis Technology](http://www.basistech.com/) has a product called the [Rosette Entity Resolver](http://www.basistech.com/text-analytics/rosette/entity-resolver/). Their technology has been used
by "Amazon.com, EMC, Endeca/Oracle, Exalead/Dassault, Fujitsu, Google, Hewlett-Packard, Microsoft, Oracle, and governments around the world". If you're a newer smaller company then you might want to check out their [startup program](http://www.basistech.com/about/startup/).

### Do it yourself

There are number of open source tools and frameworks that I've found. None of them are an Apache projects. Here's what I've found:

1. [Dedupe](https://github.com/datamade/dedupe) python package
1. [Duke](https://github.com/larsga/Duke)
1. [elasticsearch-entity-resolution](https://github.com/YannBrrd/elasticsearch-entity-resolution)
1. [Berkeley Entity Resolution](https://github.com/gregdurrett/berkeley-entity)
1. [SERF Project](http://infolab.stanford.edu/serf/)
1. [Ch. 3 of Mining Massive Datasets](http://infolab.stanford.edu/~ullman/mmds/ch3.pdf)

Of these options, Dedupe is the only one maintained by an organization ([Datamade](http://datamade.us/)). My personal experience with Duke is that it seemed to have some buzz but none of the demo examples worked for me. The elasticsearch entity resolution plugin is based on Duke (haven't personally tried it). The Berkeley ER project looks like it's Greg's thesis and appears to be a work in progress. When I contacted one of the researchers at the SERF project, they said it's no longer active. Ch.3 of Mining massive datasets included an entity resolution example that may work for certain use cases.

Let's take a closer look at *Dedupe* and Ch.3 of Mining Massive datasets (hereafter referred to as *MMDS*). See **appendix 1 & 2** for examples.

### Do some of it yourself but hire a company

One approach is to refurbish your dataset by kicking the complexity to some else's API. Depending on your
data, you may be able to take advantage of the [Google Places API](https://developers.google.com/places/). This would allow you to return standard names for locations and organizations; you may even be able to cross-reference their phone numbers. This API is considerably different than the one produced by Facebook or Foursquare because it's not user-generated; from what I can tell. See **Appendix 3** for an example.

## Appendix 0: Getting Data

Our data is from the Dedupe [examples](https://github.com/datamade/dedupe-examples).

## Appendix 1: MMDS Approach

The main moving part here is the edit distance. 

In [19]:
from nltk import edit_distance

edit_distance('foo', 'far')

2

## Appendix 2: Dedupe (Python Package)

Note that [dedupe tests](https://github.com/datamade/dedupe/tree/master/tests) includes the referenced datasets. I've modified the tests so they play well within a notebook. We will use Dedupe to solve the following problems:


1. Deduplication
1. Record linkage

See [dedupe examples](https://github.com/datamade/dedupe-examples) for examples that include mysql and postgres. 

### Deduplication

The example that I've demonstrated below are taken from these scripts. I've already trained the data; so you can just run this notebook and see results. I had to run the example outside of the notebook because `TypeError` is raised when a cell with an empty value is encountered.

In [1]:
import os
os.system("python csv_example/csv_example.py")

0

Given that this is example data, we already know which duplicates exist in the
date. Let's evaluate our procedure

In [2]:
# Evaluating output

manual_clusters = 'csv_example/csv_example_input_with_true_ids.csv'
dedupe_clusters = 'csv_example/csv_example_output.csv'

from csv_example.csv_evaluation import dupePairs, evaluateDuplicates

In [3]:
true_dupes = dupePairs(manual_clusters, 'True Id')
test_dupes = dupePairs(dedupe_clusters, 'Cluster ID')

evaluateDuplicates(test_dupes, true_dupes)

found duplicate
5634
precision
0.9774582889598864
recall
0.8333837772397095


In [4]:
import pandas as pd

dedupe_output = pd.read_csv(dedupe_clusters)
input_file = 'csv_example/csv_example_messy_input.csv'
dedupe_input = pd.read_csv(input_file)

In [5]:
dedupe_input[["Id", "Site name", "Address"]][0:5]

,Id,Site name,Address
0,0,Salvation Army - Temple / Salvation Army,1 N Ogden Ave
1,1,Salvation Army - Temple / Salvation Army,1 N Ogden Ave
2,2,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave
3,3,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave
4,4,Board Trustees-City Colleges of Chicago - Oli...,10001 S Woodlawn Ave


In [6]:
dedupe_output[["Cluster ID", "confidence_score", "Id", "Site name", "Address"]][0:5]

,Cluster ID,confidence_score,Id,Site name,Address
0,18,0.831860,0,Salvation Army - Temple / Salvation Army,1 N Ogden Ave
1,18,0.831860,1,Salvation Army - Temple / Salvation Army,1 N Ogden Ave
2,390,0.547965,2,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave
3,390,0.547965,3,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave
4,591,0.438202,4,Board Trustees-City Colleges of Chicago - Oli...,10001 S Woodlawn Ave


### Record linkage

This example is also taken from the dedupe example scripts. I had to make some small modifications for it to run but I'll try to submit those as a pull request. I've already trained the data so you can just run this notebook and see the results.

From the example script docstring:

> This code demonstrates how to use RecordLink with two comma separated
> values (CSV) files. We have listings of products from two different
> online stores. The task is to link products between the datasets.

> The output will be a CSV with our linkded results.

I've pulled out the interesting bits of the example script.

In [7]:
from __future__ import print_function
from future.builtins import next

import os
import csv
import re
import collections
#import logging
import optparse
import numpy

import dedupe
from unidecode import unidecode

# from example script
from record_linkage_example.record_linkage import (preProcess, readData, 
                                                   descriptions)

# params

output_file = 'record_linkage_example/data_matching_output.csv'
settings_file = 'record_linkage_example/data_matching_learned_settings'
training_file = 'record_linkage_example/data_matching_training.json'



In [8]:
print('importing data ...')
data_1 = readData('record_linkage_example/AbtBuy_Abt.csv')
data_2 = readData('record_linkage_example/AbtBuy_Buy.csv')


importing data ...


In [9]:
# we already know that the settings file exists
if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as sf :
        linker = dedupe.StaticRecordLink(sf)
        
else:
    raise TypeError("Notebook is not meant to be run outside example")


reading from record_linkage_example/data_matching_learned_settings


In [10]:
# ## Blocking

# ## Clustering

# Find the threshold that will maximize a weighted average of our
# precision and recall.  When we set the recall weight to 2, we are
# saying we care twice as much about recall as we do precision.
#
# If we had more data, we would not pass in all the blocked data into
# this function but a representative sample.

print('clustering...')
linked_records = linker.match(data_1, data_2, 0)

print('# duplicate sets', len(linked_records))


clustering...
# duplicate sets 359


In [11]:
# ## Writing Results

# Write our original data back out to a CSV with a new column called 
# 'Cluster ID' which indicates which records refer to each other.

cluster_membership = {}
cluster_id = None
for cluster_id, (cluster, score) in enumerate(linked_records):
    for record_id in cluster:
        cluster_membership[record_id] = (cluster_id, score)

if cluster_id :
    unique_id = cluster_id + 1
else :
    unique_id =0
    


In [12]:
with open(output_file, 'w') as f:
    writer = csv.writer(f)
    
    header_unwritten = True

    for fileno, filename in enumerate(('record_linkage_example/AbtBuy_Abt.csv', 
                                       'record_linkage_example/AbtBuy_Buy.csv')) :
        with open(filename) as f_input :
            reader = csv.reader(f_input)

            if header_unwritten :
                heading_row = next(reader)
                heading_row.insert(0, 'source file')
                heading_row.insert(0, 'Link Score')
                heading_row.insert(0, 'Cluster ID')
                writer.writerow(heading_row)
                header_unwritten = False
            else :
                next(reader)

            for row_id, row in enumerate(reader):
                cluster_details = cluster_membership.get(filename + str(row_id))
                if cluster_details is None :
                    cluster_id = unique_id
                    unique_id += 1
                    score = None
                else :
                    cluster_id, score = cluster_details
                row.insert(0, fileno)
                row.insert(0, score)
                row.insert(0, cluster_id)
                writer.writerow(row)


In [13]:
record_input1 = pd.read_csv('record_linkage_example/AbtBuy_Abt.csv')
record_input2 = pd.read_csv('record_linkage_example/AbtBuy_Buy.csv')
record_output = pd.read_csv('record_linkage_example/data_matching_output.csv')

record_input1[0:5]

,unique_id,title,description,price
0,1,Linksys EtherFast 8-Port 10/100 Switch - EZXS88W,Linksys EtherFast 8-Port 10/100 Switch - EZXS8...,$44.00
1,2,Linksys EtherFast10/100 5-Port Auto-Sensing Sw...,Linksys EtherFast10/100 5-Port Auto-Sensing Sw...,$29.00
2,3,Netgear ProSafe 5 Port 10/100 Desktop Switch -...,Netgear ProSafe 5 Port 10/100 Desktop Switch -...,$40.00
3,4,Belkin F3H982-10 Pro Series High Integrity 10 ...,Belkin F3H982-10 Pro Series High Integrity 10 ...,NaN
4,5,Netgear Prosafe 16 Port 10/100 Rackmount Switc...,Netgear Prosafe 16 Port 10/100 Rackmount Switc...,$131.00


In [14]:
record_input2[0:5]

,unique_id,title,description,price
0,1,Linksys EtherFast EZXS88W Ethernet Switch - EZ...,Linksys EtherFast 8-Port 10/100 Switch (New/Wo...,NaN
1,2,Linksys EtherFast EZXS55W Ethernet Switch,5 x 10/100Base-TX LAN,NaN
2,3,Netgear ProSafe FS105 Ethernet Switch - FS105NA,NETGEAR FS105 Prosafe 5 Port 10/100 Desktop Sw...,NaN
3,4,Belkin Pro Series High Integrity VGA/SVGA Moni...,1 x HD-15 - 1 x HD-15 - 10ft - Beige,NaN
4,5,Netgear ProSafe JFS516 Ethernet Switch,Netgear ProSafe 16 Port 10/100 Rackmount Switc...,NaN


In [15]:
record_output[0:5]

,Cluster ID,Link Score,source file,unique_id,title,description,price
0,189,0.999620,0,1,Linksys EtherFast 8-Port 10/100 Switch - EZXS88W,Linksys EtherFast 8-Port 10/100 Switch - EZXS8...,$44.00
1,359,NaN,0,2,Linksys EtherFast10/100 5-Port Auto-Sensing Sw...,Linksys EtherFast10/100 5-Port Auto-Sensing Sw...,$29.00
2,360,NaN,0,3,Netgear ProSafe 5 Port 10/100 Desktop Switch -...,Netgear ProSafe 5 Port 10/100 Desktop Switch -...,$40.00
3,208,0.998993,0,4,Belkin F3H982-10 Pro Series High Integrity 10 ...,Belkin F3H982-10 Pro Series High Integrity 10 ...,NaN
4,361,NaN,0,5,Netgear Prosafe 16 Port 10/100 Rackmount Switc...,Netgear Prosafe 16 Port 10/100 Rackmount Switc...,$131.00


It's not as obvious what the record linkage procedure is doing. Given that we're working with example data, 
unique id's were already present so we can see which one's are correct. We can see in the next sell that the
unique id's match up.

In [16]:
record_input1.count()['unique_id'] + record_input2.count()['unique_id'] \
== record_output.count()['unique_id']

True

## Appendix 3: Google Places API

Please verify with Google that this is not a violation of their terms of service before implementing this approach.
